Использует полученные из SAbDab файл structures_summary.tsv с метаинформацией о структурах и структуры в формате pdb с нумерацией по системе Chothia в папке structures/chothia

## Метаинформация

In [5]:
import pandas as pd

In [6]:
metainf = pd.read_csv('structures_summary.tsv', sep='\t',
                     na_values={'resolution': ['NOT', 'NA', 'None', '0'],
                                'r_free': ['NOT', 'NA', 'None', 'unknown'],
                                'affinity': ['NOT', 'NA', 'None'],
                                'delta_g': ['NOT', 'NA', 'None'],
                                'affinity_method': ['NOT', 'NA', 'None'],
                                'temperature': ['NOT', 'NA', 'None'],
                                'pmid': ['NOT', 'NA', 'None'],
                               },
                      
                     )
metainf['resolution'] = metainf['resolution'].str.extract('([^,]+)').astype(float)
metainf['date'] = pd.to_datetime(metainf['date'].astype(str), 
                                 format="%m/%d/%y", 
                                 errors='coerce')

In [11]:
# это, кажется, проблема разметки цепей. Их мало, исключу
metainf.dropna(subset=['Hchain', 'Lchain'], inplace=True)

In [9]:
# разделим антигены на Fv
for col in ['antigen_chain', 'antigen_type', 'antigen_het_name', 'antigen_name']:
    metainf[col] = metainf[col].str.split(pat=' | ', regex=False)

In [11]:
metainf

,pdb,Hchain,Lchain,model,antigen_chain,antigen_type,antigen_het_name,antigen_name,short_header,date,...,scfv,engineered,heavy_subclass,light_subclass,light_ctype,affinity,delta_g,affinity_method,temperature,pmid
0,7sue,F,E,0,[J],[protein],NaN,[nucleoprotein],VIRAL PROTEIN,2022-08-10,...,False,True,IGHV1,IGLV2,Lambda,NaN,NaN,NaN,NaN,NaN
1,7sue,I,G,0,[K],[protein],NaN,[nucleoprotein],VIRAL PROTEIN,2022-08-10,...,False,True,IGHV1,IGLV2,Lambda,NaN,NaN,NaN,NaN,NaN
2,7sue,B,A,0,[D],[protein],NaN,[nucleoprotein],VIRAL PROTEIN,2022-08-10,...,False,True,IGHV1,IGLV2,Lambda,NaN,NaN,NaN,NaN,NaN
3,7sue,H,L,0,[C],[protein],NaN,[nucleoprotein],VIRAL PROTEIN,2022-08-10,...,False,True,IGHV1,IGLV2,Lambda,NaN,NaN,NaN,NaN,NaN
4,5js9,E,F,0,"[C, D]","[protein, protein]","[NA, NA]","[gp120, gp41]",VIRAL PROTEIN/IMMUNE SYSTEM,2016-07-13,...,False,True,unknown,IGKV1,Kappa,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5267,4d9q,H,L,0,[A],[protein],NaN,[factor d],HYDROLASE/IMMUNE SYSTEM,2012-01-11,...,False,True,IGHV7,IGKV1,Kappa,3.000000e-11,-14.355775,SPR,NaN,TBD
5268,6vtt,H,L,0,"[G, F, E]","[protein, protein, protein]","[NA, NA, NA]","[envelope glycoprotein gp120, envelope glycopr...",VIRAL PROTEIN/IMMUNE SYSTEM,2020-04-08,...,False,True,IGHV3,IGLV1,Lambda,NaN,NaN,NaN,NaN,NaN
5269,6vtu,H,L,0,[A],[carbohydrate],[MAN],[ALPHA-D-MANNOSE],IMMUNE SYSTEM,2020-02-13,...,False,True,IGHV4,IGLV2,Lambda,NaN,NaN,NaN,NaN,NaN
5270,6azm,B,A,0,[E],[peptide],NaN,[circumsporozoite protein nanp 5-mer],IMMUNE SYSTEM,2017-09-11,...,False,True,IGHV3,IGKV4,Kappa,NaN,NaN,NaN,NaN,NaN


## Извлечение CDR

### Остатки на позициях

In [13]:
import sys
stdout = sys.stdout

import __main__
__main__.pymol_argv = [ 'pymol', '-c']
import pymol 
from pymol import cmd
pymol.finish_launching()

sys.stdout = stdout

 Detected 8 CPU cores.  Enabled multithreaded rendering.


In [14]:
CDR_num = {
    'L1': (24, 34),
    'L2': (50, 56),
    'L3': (89, 97),
    'H1': (26, 32),
    'H2': (52, 56),
    'H3': (95, 102),
}

In [15]:
# оставила по одному представителю из каждого pdb, потому что обычно в структуре присутсвуют копии с одной последовательностью 
metainf_first = metainf.drop_duplicates(subset=['pdb'])

In [16]:
metainf_first

,pdb,Hchain,Lchain,model,antigen_chain,antigen_type,antigen_het_name,antigen_name,short_header,date,...,scfv,engineered,heavy_subclass,light_subclass,light_ctype,affinity,delta_g,affinity_method,temperature,pmid
0,7sue,F,E,0,J,protein,NaN,nucleoprotein,VIRAL PROTEIN,2022-08-10,...,False,True,IGHV1,IGLV2,Lambda,NaN,NaN,NaN,NaN,NaN
4,5js9,E,F,0,C | D,protein | protein,NA | NA,gp120 | gp41,VIRAL PROTEIN/IMMUNE SYSTEM,2016-07-13,...,False,True,unknown,IGKV1,Kappa,NaN,NaN,NaN,NaN,NaN
6,6lfm,S,S,0,A | B,protein | protein,NA | NA,guanine nucleotide-binding protein g(i) subuni...,MEMBRANE PROTEIN,2019-12-03,...,True,True,unknown,unknown,unknown,NaN,NaN,NaN,NaN,NaN
7,6lfo,S,S,0,A | C | B,protein | protein | protein,NA | NA | NA,guanine nucleotide-binding protein g(i) subuni...,MEMBRANE PROTEIN,2019-12-03,...,True,True,unknown,unknown,unknown,NaN,NaN,NaN,NaN,NaN
8,7su1,H,L,0,C,protein,NaN,cytotoxic t-lymphocyte protein 4,IMMUNE SYSTEM,2022-03-02,...,False,True,IGHV3,IGKV3,Kappa,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5265,5ock,H,L,0,A,peptide,NaN,cep1 peptide (from enolase),IMMUNE SYSTEM,2017-07-03,...,False,True,IGHV4,IGLV1,Lambda,NaN,NaN,NaN,NaN,NaN
5266,4d9q,E,D,0,B,protein,NaN,factor d,HYDROLASE/IMMUNE SYSTEM,2012-01-11,...,False,True,IGHV7,IGKV1,Kappa,3.000000e-11,-14.355775,SPR,NaN,TBD
5268,6vtt,H,L,0,G | F | E,protein | protein | protein,NA | NA | NA,envelope glycoprotein gp120 | envelope glycopr...,VIRAL PROTEIN/IMMUNE SYSTEM,2020-04-08,...,False,True,IGHV3,IGLV1,Lambda,NaN,NaN,NaN,NaN,NaN
5269,6vtu,H,L,0,A,carbohydrate,MAN,ALPHA-D-MANNOSE,IMMUNE SYSTEM,2020-02-13,...,False,True,IGHV4,IGLV2,Lambda,NaN,NaN,NaN,NaN,NaN


In [18]:
CDR_df = pd.DataFrame({'pdb': metainf_first['pdb']})

In [19]:
CDR_df.set_index('pdb', inplace=True)

In [1]:
from tqdm import tqdm

In [ ]:
for row in tqdm(list(metainf_first.iterrows())):
    cmd.load(f'structures/chothia/{row[1]["pdb"]}.pdb', 'current')
    myspace = dict()
    for chain in ('L', 'H'):
        for num in (1, 2, 3):
            
            start, end = CDR_num[f"{chain}{num}"]
            cmd.select(f'CDR_{chain}{num}', 
                       f'current and chain {row[1][f"{chain}chain"]} and resi {start}-{end}')
            
            myspace[f'CDR_{chain}{num}_resids'] = []
            
            cmd.iterate(f'CDR_{chain}{num} and name CA',
                        f'CDR_{chain}{num}_resids.append((resi, oneletter ))',
                       space=myspace)
            
            for resid, resname in myspace[f'CDR_{chain}{num}_resids']:
                CDR_df.loc[row[1]["pdb"], f'{chain}{num}_{resid}'] = resname
    cmd.delete('*')

In [23]:
CDR_df = CDR_df.copy()  # иначе слишком фрагментированная из-за постоянного добавления столбцов 

In [29]:
CDR_df = CDR_df.fillna('')

In [33]:
CDR_df = CDR_df[sorted(CDR_df.columns)]

In [34]:
CDR_df

,H1_26,H1_27,H1_28,H1_29,H1_30,H1_30A,H1_30B,H1_31,H1_31A,H1_31B,...,L3_95C,L3_95D,L3_95E,L3_95F,L3_95G,L3_95H,L3_95I,L3_95J,L3_96,L3_97
pdb,,,,,,,,,,,,,,,,,,,,,
7sue,,,T,F,S,,,S,,,...,,,,,,,,,Y,V
5js9,G,V,N,T,F,,,G,L,,...,,,,,,,,,G,T
6lfm,G,F,A,F,S,,,S,,,...,,,,,,,,,I,Y
6lfo,G,F,A,F,S,,,S,,,...,,,,,,,,,I,Y
7su1,G,F,T,F,S,,,H,,,...,,,,,,,,,W,T
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5ock,G,F,P,M,S,,,E,S,,...,,,,,,,,,F,V
4d9q,G,Y,T,F,T,,,N,,,...,,,,,,,,,Y,T
6vtt,Q,F,R,F,D,,,G,,,...,A,,,,,,,,R,V


### Объединение в участки по отдельным CDR

In [46]:
CDR_seq = pd.DataFrame({'pdb': metainf_first['pdb']}).set_index('pdb')

In [48]:
for chain in ('L', 'H'):
    for num in (1, 2, 3):
        columns = CDR_df.columns[CDR_df.columns.str.startswith(f'{chain}{num}')]
        CDR_seq[f'{chain}{num}'] = \
                CDR_df[columns].apply(
                                        lambda x: ''.join(x),
                                        axis=1
                                    )

In [4]:
CDR_seq

,pdb,L1,L2,L3,H1,H2,H3
0,7sue,TGTSSDVGGYNYVS,EVTNRPS,SSYTSSSLYV,TFSSC,IPILGI,SGSYYRTDYYYYAMDVGWEFG
1,5js9,RASQSITGNWVA,RGAALLG,QQYDTYPGT,GVNTFGLY,WRWK,KWSGLHHDGVMAFSSTSTYD
2,6lfm,ASGFAFSSFGM,YISSGSGT,MYYCVRSIY,GFAFSSF,SSGSGT,GSSPFDFSIYYY
3,6lfo,ASGFAFSSFGM,YISSGSGT,MYYCVRSIY,GFAFSSF,SSGSGT,GSSPFDFSIYYY
4,7su1,RASQEVGESELA,GAFSRAT,QQYGSSPWT,GFTFSHY,SYDGHN,PFDYHGWLG
...,...,...,...,...,...,...,...
2419,5ock,SGDDSILRSAFVS,DDRQRPS,AAWNGRLSAFV,GFPMSESY,IHTGT,SNWLDPIRGGS
2420,4d9q,ITSTDIDDDMN,GGNTLRP,LQSDSLPYT,GYTFTNY,NTYTGE,NNEGGV
2421,6vtt,SGNTSNIGNNFVS,ETDKRPS,ATWAASLSSARV,QFRFDGY,SHDGI,ECEEWWSDDFGKQLPCAKSRGGLVGIADNDLRED
2422,6vtu,TGTSSDIGQYNSVS,GVTSRPS,SSHADENMAL,GVSINDGY,FGRSGN,FAAYNSLDVGMEGL


In [71]:
CDR_seq.to_csv('CDR_seq_df.csv')